In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline , make_pipeline

In [3]:
titanic = pd.read_csv(r"E:\Jupyter Notebooks\train.csv")
titanic.drop(['PassengerId','Name','Ticket','Cabin'],axis = 1, inplace = True)

In [4]:
titanic

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


## Train test split

In [6]:
xtrain,xtest,ytrain,ytest = train_test_split(titanic.drop(columns=['Survived']),
                                                 titanic['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

In [15]:
xtrain

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [27]:
ytrain

331    0
733    0
382    0
704    0
813    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: Survived, Length: 712, dtype: int64

## Preprocessing transformers

 simple imputer
trans1  = ColumnTransformer([
        ('age', SimpleImputer(), ["Age"]),
    ('embark',SimpleImputer(strategy = 'most_frequent'), ["Embarked"])
]
    , remainder = 'passthrough'
)

In [14]:
# instead of the above format use indexes instead of column names since aage the code will not 
# work since we are passing data as a numpy array which doesn't have feature names

In [17]:
#simple imputer 
trans1 = ColumnTransformer([ ('age', SimpleImputer(), [2]),
                                    ('embark',SimpleImputer(strategy = 'most_frequent'), [6]) ], 
                                          remainder = 'passthrough' )

In [18]:
# OHE
trans2  = ColumnTransformer([
        ('sex', OneHotEncoder(sparse = False, handle_unknown = 'ignore'), [1]),
    ('embark',OneHotEncoder(sparse = False, handle_unknown = 'ignore'), [6])
]
    , remainder = 'passthrough'
)

In [22]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [19]:
# classifier
trans3  = DecisionTreeClassifier()


## Pipeline

In [23]:
pipe = Pipeline([
    ('trans1', trans1),
    ('trans2', trf2),
    ('trans3', trans3)
])

## Pipeline (easier to debug) Vs make_pipeline 
Pipeline requires naming of steps, make_pipeline does not.

(Same applies to ColumnTransformer vs make_column_transformer)

## Alternate Syntax
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [24]:
pipe.fit(xtrain, ytrain)

ValueError: could not convert string to float: 'male'

In [26]:
# chatgpt code
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Ensure xtrain is a DataFrame
if not isinstance(xtrain, pd.DataFrame):
    xtrain = pd.DataFrame(xtrain)

# Ensure xtest is a DataFrame if you plan to transform it later
if not isinstance(xtest, pd.DataFrame):
    xtest = pd.DataFrame(xtest)

# Define combined transformers
preprocessor = ColumnTransformer([
        ('age', SimpleImputer(), ["Age"]),
        ('embark_impute', SimpleImputer(strategy='most_frequent'), ["Embarked"]),
        ('sex', OneHotEncoder(sparse=False, handle_unknown='ignore'), ["Sex"]),
        ('embark_encode', OneHotEncoder(sparse=False, handle_unknown='ignore'), ["Embarked"])
    ], remainder='passthrough'
)

# Create pipeline with a final estimator
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])

# Fit pipeline
pipe.fit(xtrain, ytrain)


ValueError: could not convert string to float: 'S'